In [3]:
#  Step 1: Import libraries
import pandas as pd
import os


In [4]:
# Step 2: Define paths
RAW_PATH = 'data/raw/'
OUTPUT_PATH = 'data/processed/combined_disaster_data.csv'


In [5]:
#  Step 3: Load and prepare Flood dataset
flood = pd.read_csv(os.path.join(RAW_PATH, 'flood.csv'))
flood['FloodOccurred'] = flood['FloodProbability'].apply(lambda x: 1 if x > 0.5 else 0)
flood = flood.drop(columns=['FloodProbability'])
flood['disaster_type'] = flood['FloodOccurred'].apply(lambda x: 'Flood' if x == 1 else 'NoDisaster')
flood = flood.drop(columns=['FloodOccurred'])


In [7]:
# Step 4: Load and prepare Earthquake dataset
earthquake = pd.read_csv(os.path.join(RAW_PATH, 'earthquake.csv'))
earthquake = earthquake[['Latitude', 'Longitude', 'Depth', 'Magnitude']]
earthquake['disaster_type'] = 'Earthquake'


In [8]:
# Step 5: Load Landslide dataset
landslide = pd.read_csv(os.path.join(RAW_PATH, 'landslide.csv'))
landslide['disaster_type'] = 'Landslide'


In [9]:
#  Step 6: Load Cyclone dataset
cyclone = pd.read_csv(os.path.join(RAW_PATH, 'cyclone.csv'))
cyclone['disaster_type'] = cyclone['Cyclone'].apply(lambda x: 'Cyclone' if x == 1 else 'NoDisaster')
cyclone = cyclone.drop(columns=['Cyclone'])


In [10]:
#  Step 7: Standardize all column names
flood.columns = flood.columns.str.lower()
earthquake.columns = earthquake.columns.str.lower()
landslide.columns = landslide.columns.str.lower()
cyclone.columns = cyclone.columns.str.lower()


In [11]:
#  Step 8: Pad missing columns
flood_cols = set(flood.columns)
earthquake_cols = set(earthquake.columns)
landslide_cols = set(landslide.columns)
cyclone_cols = set(cyclone.columns)
all_columns = list(flood_cols | earthquake_cols | landslide_cols | cyclone_cols)

def pad_columns(df):
    for col in all_columns:
        if col not in df.columns:
            df[col] = None
    return df[all_columns]

flood = pad_columns(flood)
earthquake = pad_columns(earthquake)
landslide = pad_columns(landslide)
cyclone = pad_columns(cyclone)


In [12]:
#  Step 9: Combine and save
combined = pd.concat([flood, earthquake, landslide, cyclone], ignore_index=True)
os.makedirs('data/processed', exist_ok=True)
combined.to_csv(OUTPUT_PATH, index=False)
print(f" Combined dataset saved at: {OUTPUT_PATH}")


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16364\3752539293.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat([flood, earthquake, landslide, cyclone], ignore_index=True)


 Combined dataset saved at: data/processed/combined_disaster_data.csv


In [13]:
# Remove completely empty columns across all DataFrames
def drop_all_na_cols(df):
    return df.dropna(axis=1, how='all')

flood = drop_all_na_cols(flood)
earthquake = drop_all_na_cols(earthquake)
landslide = drop_all_na_cols(landslide)
cyclone = drop_all_na_cols(cyclone)


In [14]:
combined = pd.concat([flood, earthquake, landslide, cyclone], ignore_index=True)


In [4]:
import pandas as pd
import numpy as np

# Load your combined dataset
df = pd.read_csv("data/processed/combined_disaster_data.csv")

# Define realistic value ranges for each relevant column
realistic_ranges = {
    'proximity_to_water': (0, 10),             # in km
    'earthquake_activity': (0, 10),            # risk score
    'soil_type_sand': (0, 1),                  # binary
    'soil_saturation': (0, 100),               # percentage
    'rainfall_mm': (0, 500),                   # mm
    'soil_type_silt': (0, 1),                  # binary
    'proximity_to_coastline': (0, 50),         # km
    'latitude': (-90, 90),                     # degrees
    'pre_existing_disturbance': (0, 10),       # score
    'humidity': (0, 100),                      # percentage
    'wind_shear': (0, 30),                     # m/s
    'vorticity': (-5, 5),                      # arbitrary scale
    'ocean_depth': (0, 11000),                 # meters
    'sea_surface_temperature': (0, 35),        # °C
    'slope_angle': (0, 90),                    # degrees
    'longitude': (-180, 180),                  # degrees
    'soil_type_gravel': (0, 1),                # binary
    'vegetation_cover': (0, 10),               # index
    'landslide': (0, 1),                       # binary
    'depth': (0, 700),                         # km
    'magnitude': (0, 10)                       # Richter scale
}

# Fill in empty or missing columns
for col, (min_val, max_val) in realistic_ranges.items():
    if col in df.columns:
        is_empty = df[col].isna() | (df[col].astype(str).str.strip() == '')
        if is_empty.all():
            df[col] = np.random.uniform(min_val, max_val, size=len(df)).round(2)
        elif is_empty.any():
            df.loc[is_empty, col] = np.random.uniform(min_val, max_val, size=is_empty.sum()).round(2)
    else:
        df[col] = np.random.uniform(min_val, max_val, size=len(df)).round(2)

# Save cleaned dataset (overwrite existing)
df.to_csv("data/processed/combined_disaster_data.csv", index=False)

print(" Cleaned dataset saved with realistic values at: data/processed/combined_disaster_data.csv")


 Cleaned dataset saved with realistic values at: data/processed/combined_disaster_data.csv
